In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
text = '2025 will be ours'

encoding = tokenizer.encode_plus(text, add_special_tokens=True, padding='max_length', truncation=True, 
                                 return_attention_mask = True, return_tensors='pt')

encoding

In [ ]:
input = encoding['input_ids'][0]
attention_mask = encoding['attention_mask'][0]
attention_mask

In [69]:
from transformers import BertTokenizer, BertForMaskedLM, BertForNextSentencePrediction, BertForQuestionAnswering
from torch.nn import functional as F
import torch

In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [17]:
model = BertForMaskedLM.from_pretrained('bert-base-uncased', return_dict=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [55]:
text = "The Indian city known for the Taj Mahal is " + tokenizer.mask_token + "."

encoding = tokenizer.encode_plus(text, return_tensors='pt')

In [56]:
mask_index = torch.where(encoding['input_ids'][0] == tokenizer.mask_token_id)

In [57]:
# Get model output
with torch.no_grad():
    output = model(**encoding)
logits = output.logits

In [58]:
softmax = F.softmax(logits, dim=-1)

In [59]:
mask_word = softmax[0, mask_index, :]

In [60]:
top_10 = torch.topk(mask_word, 10, dim=1)[1][0]

In [61]:
for token in top_10:
    word = tokenizer.decode([token], skip_special_tokens=True)
    new_sentence = text.replace(tokenizer.mask_token, word)
    print(new_sentence)

The Indian city known for the Taj Mahal is mumbai.
The Indian city known for the Taj Mahal is delhi.
The Indian city known for the Taj Mahal is bangalore.
The Indian city known for the Taj Mahal is pune.
The Indian city known for the Taj Mahal is jaipur.
The Indian city known for the Taj Mahal is kolkata.
The Indian city known for the Taj Mahal is hyderabad.
The Indian city known for the Taj Mahal is chennai.
The Indian city known for the Taj Mahal is agra.
The Indian city known for the Taj Mahal is ahmedabad.


In [64]:
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

In [65]:
prompt = 'I came back from clg at evening'
next_sentence = 'I started to watch webseries'

In [67]:
encoding = tokenizer.encode_plus(prompt, next_sentence, return_tensors='pt')
outputs = model(**encoding)[0]
softmax = F.softmax(outputs, dim=1)

print(softmax)

tensor([[9.9987e-01, 1.2635e-04]], grad_fn=<SoftmaxBackward0>)


## Quetion answering

In [78]:
tokenizer = BertTokenizer.from_pretrained('deepset/bert-base-uncased-squad2')
model = BertForQuestionAnswering.from_pretrained('deepset/bert-base-uncased-squad2')

tokenizer_config.json:   0%|          | 0.00/302 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [84]:
Question = "What is the historical significance of the Taj Mahal, and who was responsible for its construction?"
context = """The Taj Mahal, one of the most famous monuments in the world, was commissioned in 1632 by the Mughal Emperor Shah Jahan 
in memory of his favorite wife, Mumtaz Mahal. Located in Agra, India, it stands as a symbol of eternal love and is recognized for its 
stunning Mughal architecture. The construction took approximately 20 years, employing thousands of artisans and laborers. 
Its white marble structure is adorned with intricate carvings, calligraphy, and semi-precious stones. 
The Taj Mahal is a UNESCO World Heritage Site and attracts millions of tourists every year. 
It represents the artistic and cultural zenith of the Mughal Empire and is often considered one of the New Seven Wonders of the World."""


In [85]:
tokenized_input = tokenizer(Question, context, return_tensors='pt')
tokenized_input

{'input_ids': tensor([[  101,  2054,  2003,  1996,  3439,  7784,  1997,  1996, 11937,  3501,
         27913,  1010,  1998,  2040,  2001,  3625,  2005,  2049,  2810,  1029,
           102,  1996, 11937,  3501, 27913,  1010,  2028,  1997,  1996,  2087,
          3297, 10490,  1999,  1996,  2088,  1010,  2001,  4837,  1999, 28216,
          2011,  1996, 17877,  3750,  7890, 14855,  4819,  1999,  3638,  1997,
          2010,  5440,  2564,  1010, 12954,  2696,  2480, 27913,  1012,  2284,
          1999, 29542,  1010,  2634,  1010,  2009,  4832,  2004,  1037,  6454,
          1997, 10721,  2293,  1998,  2003,  3858,  2005,  2049, 14726, 17877,
          4294,  1012,  1996,  2810,  2165,  3155,  2322,  2086,  1010, 15440,
          5190,  1997, 26818,  1998, 23428,  1012,  2049,  2317,  7720,  3252,
          2003, 19189,  2007, 17796, 22838,  1010,  2655, 23132,  1010,  1998,
          4100,  1011,  9062,  6386,  1012,  1996, 11937,  3501, 27913,  2003,
          1037, 12239,  2088,  4348,  

In [86]:
with torch.no_grad():
    outputs = model(**tokenized_input)

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = tokenized_input.input_ids[0, answer_start_index:answer_end_index+1]
tokenizer.decode(predict_answer_tokens)


'mumtaz mahal'